In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
204,AFG,Asia,Afghanistan,2020-08-01,36710.0,168.0,1283.0,12.0,943.015,4.316,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
350,ALB,Europe,Albania,2020-08-01,5276.0,79.0,157.0,3.0,1833.345,27.452,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
560,DZA,Africa,Algeria,2020-08-01,30394.0,563.0,1210.0,10.0,693.119,12.839,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-31,France,187809.0,0.0,30289.0,0.0,0.0,0.0
0,2020-07-31,China,87683.0,0.0,4667.0,0.0,0.0,0.0
0,2020-07-31,Italy,247544.0,0.0,35161.0,0.0,0.0,0.0
0,2020-07-31,Spain,287242.0,0.0,28457.0,0.0,0.0,0.0
0,2020-07-31,United States,4569385.0,0.0,153441.0,0.0,0.0,0.0
0,2020-07-31,World,17575183.0,0.0,680290.0,0.0,0.0,0.0
0,2020-07-31,United Kingdom,303274.0,0.0,46126.0,0.0,0.0,0.0
0,2020-07-31,Germany,207942.0,0.0,9142.0,0.0,0.0,0.0
0,2020-07-31,Iran,305058.0,0.0,16764.0,0.0,0.0,0.0
0,2020-07-31,Turkey,230781.0,0.0,5687.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-01,France,187809.0,187919.0,30289.0,30265.0,-110.0,24.0
0,2020-08-01,China,87683.0,87655.0,4667.0,4661.0,28.0,6.0
0,2020-08-01,Italy,247544.0,247537.0,35161.0,35141.0,7.0,20.0
0,2020-07-31,Spain,287242.0,288522.0,28457.0,28445.0,-1280.0,12.0
0,2020-08-01,United States,4569385.0,4562037.0,153441.0,153314.0,7348.0,127.0
0,2020-08-01,World,17575183.0,17579197.0,680290.0,679505.0,-4014.0,785.0
0,2020-08-01,United Kingdom,303274.0,303181.0,46126.0,46119.0,93.0,7.0
0,2020-08-01,Germany,207942.0,209653.0,9142.0,9148.0,-1711.0,-6.0
0,2020-08-01,Iran,305058.0,304204.0,16764.0,16766.0,854.0,-2.0
0,2020-08-01,Turkey,230781.0,230873.0,5687.0,5691.0,-92.0,-4.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")